In [1]:
import torch
import triton
import triton.language as tl
import time

In [2]:
def is_cuda():
    return triton.runtime.driver.active.get_current_target().backend == "cuda"

In [3]:
def is_hip_mi200():
    target = triton.runtime.driver.active.get_current_target()
    return target.backend == 'hip' and target.arch == 'gfx90a'

In [72]:
"""
PA2 Part 2: MatMul+Relu+Add Fused Optimization.
The kernel uses several optimization techniques:

  1. Shared memory tiling.
  2. Register tiling.
  3. Cooperative fetching.
  4. Operator Fusion
  5. Write cache / epilogue fusion.

Fill in the missing parts (marked with TODO).
"""

# -----------------------------------------------------------------------------
# Tiling parameters - You will need to change these to achieve better results.
# -----------------------------------------------------------------------------
BLOCK_M = 128 # Tile size in the M dimension.
BLOCK_N = 128 # Tile size in the N dimension.
BLOCK_K = 64 # Tile size in the K dimension.


# -----------------------------------------------------------------------------
# Triton Kernel: Matrix Multiplication + ReLU + Add
#
# The kernel uses:
#   Step 1: Tile assignment (each kernel computes a tile of C)
#   Step 2: Shared memory tiling + Cooperative Fetching: Load tiles of A and B.
#   Step 3: Register tiling: Use a register accumulator.
#   Step 4: Add and ReLU fusion
#   Step 5: Write cache/Epilogue: Write the final tile back to global memory.
# -----------------------------------------------------------------------------
@triton.jit
def matmul_add_relu_kernel_fp16(
    a_ptr, b_ptr, c_ptr, d_ptr,
    M: tl.constexpr, N: tl.constexpr, K: tl.constexpr,
    stride_am: tl.constexpr, stride_ak: tl.constexpr,
    stride_bk: tl.constexpr, stride_bn: tl.constexpr,
    stride_cm: tl.constexpr, stride_cn: tl.constexpr,
    stride_dm: tl.constexpr, stride_dn: tl.constexpr,
    BLOCK_M: tl.constexpr, BLOCK_N: tl.constexpr, BLOCK_K: tl.constexpr,
):
    # -------------------------------------------------------------------------
    # Step 1: Tile: Assignment
    #
    # Each kernel instance is mapped to a tile in the output matrix C.
    # Compute the starting indices (m_start, n_start) for this tile.
    # -------------------------------------------------------------------------
    # Compute the tile indices using program_id(0) for M and program_id(1) for N
    pid_m = tl.program_id(0)
    pid_n = tl.program_id(1)

    # Calculate the starting indices for this tile
    m_start = pid_m * BLOCK_M
    n_start = pid_n * BLOCK_N

    # Create offsets for each thread within the block
    offs_m = m_start + tl.arange(0, BLOCK_M)
    offs_n = n_start + tl.arange(0, BLOCK_N)


    mask_m = offs_m < M # to prevent out of boundary errors
    mask_n = offs_n < N

    # -------------------------------------------------------------------------
    # Step 2: Register Tiling
    # -------------------------------------------------------------------------
    # Initialize the accumulator with zeros
    acc = tl.zeros((BLOCK_M, BLOCK_N), dtype=tl.float32)

    # -------------------------------------------------------------------------
    # Step 3: Shared Memory Tiling & Cooperative Fetching.
    # Compute pointers to the sub-tiles of A and B that are needed to compute
    # the current C tile. The offsets here serve to load BLOCK_SIZE_M x BLOCK_SIZE_K
    # and BLOCK_SIZE_K x BLOCK_SIZE_N blocks from A and B respectively.
    # -------------------------------------------------------------------------
    for k in range(0, K, BLOCK_K):
        # Create offsets and masks for k dimension
        offs_k = k + tl.arange(0, BLOCK_K)
        mask_k = offs_k < K

        # Load blocks from matrices A and B
        a_ptrs = a_ptr + (offs_m[:, None] * stride_am + offs_k[None, :] * stride_ak)
        b_ptrs = b_ptr + (offs_k[:, None] * stride_bk + offs_n[None, :] * stride_bn)

        # Again tl.load has masks because of boundary conditions
        a = tl.load(a_ptrs, mask=mask_m[:, None] & mask_k[None, :], other=0.0)
        b = tl.load(b_ptrs, mask=mask_k[:, None] & mask_n[None, :], other=0.0)

        # Perform matrix multiplication for this block
        acc += tl.dot(a, b)

    # -------------------------------------------------------------------------
    # Step 4: Apply ReLU and Add C to the accumulator
    # -------------------------------------------------------------------------
    # Load the bias/residual values from matrix C
    c_ptrs = c_ptr + (offs_m[:, None] * stride_cm + offs_n[None, :] * stride_cn)
    c = tl.load(c_ptrs, mask=mask_m[:, None] & mask_n[None, :], other=0.0)

    # Add the bias/residual and apply ReLU
    acc = acc + c
    acc = tl.maximum(acc, 0.0)  # ReLU activation

    # -------------------------------------------------------------------------
    # Step 5: Write Cache / Epilogue Fusion: Write the computed tile to D.
    # -------------------------------------------------------------------------
    # Write the result to matrix D
    d_ptrs = d_ptr + (offs_m[:, None] * stride_dm + offs_n[None, :] * stride_dn)
    tl.store(d_ptrs, acc, mask=mask_m[:, None] & mask_n[None, :])

In [73]:
def matmul_add_relu_fp16(a: torch.Tensor, b: torch.Tensor, c: torch.Tensor) -> torch.Tensor:
    """
    Computes Output = ReLU(A @ B + C) using fp16 precision for maximum throughput.
    """
    M, K = a.shape
    K2, N = b.shape
    assert K == K2, "Incompatible dimensions"

    d = torch.empty((M, N), device=a.device, dtype=torch.float16)
    # Create launch grid
    grid = (triton.cdiv(M, BLOCK_M), triton.cdiv(N, BLOCK_N))

    matmul_add_relu_kernel_fp16[grid](
        a, b, c, d,
        M, N, K,
        a.stride(0), a.stride(1),
        b.stride(0), b.stride(1),
        c.stride(0), c.stride(1),
        d.stride(0), d.stride(1),
        BLOCK_M=BLOCK_M, BLOCK_N=BLOCK_N, BLOCK_K=BLOCK_K
    )
    return d

In [74]:
# Reference implementation using PyTorch
def reference_matmul_add_relu(A, B, C):
    result = torch.matmul(A, B).add(C).relu_()
    return result

In [75]:
# -----------------------------------------------------------------------------
# Accuracy Tests
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    torch.manual_seed(0)
    a = torch.randn((512, 512), device=torch.device("cuda"), dtype=torch.float16)
    b = torch.randn((512, 512), device=torch.device("cuda"), dtype=torch.float16)
    c = torch.randn((512, 512), device=torch.device("cuda"), dtype=torch.float16)
    triton_output = matmul_add_relu_fp16(a, b, c)
    torch_output = reference_matmul_add_relu(a, b, c)
    print(f"triton_output_with_fp16_inputs={triton_output}")
    print(f"torch_output_with_fp16_inputs={torch_output}")
    rtol = 1e-2 if is_hip_mi200() else 0.032
    if torch.allclose(triton_output, torch_output, atol=0.15, rtol=rtol):
        print("✅ Triton and Torch match")
    else:
        diff = triton_output - torch_output
        abs_diff = torch.abs(diff)
        max_abs_diff = torch.max(abs_diff)
        print(f"❌ Triton and Torch differ: {max_abs_diff=}")

triton_output_with_fp16_inputs=tensor([[ 0.0000,  6.1250,  0.0000,  ..., 10.0391,  0.0000,  0.0000],
        [ 7.9102, 15.6250, 26.6250,  ..., 11.4531,  5.3945, 18.6562],
        [ 2.7285,  0.0000,  0.0000,  ...,  0.0000, 26.1250,  0.0000],
        ...,
        [ 0.4316, 75.2500,  0.0000,  ..., 26.2812,  0.0000,  0.0000],
        [ 6.9609,  1.1260,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [27.6406, 26.9375, 22.9375,  ..., 13.5625,  6.0352, 21.6406]],
       device='cuda:0', dtype=torch.float16)
torch_output_with_fp16_inputs=tensor([[ 0.0000,  6.1289,  0.0000,  ..., 10.0391,  0.0000,  0.0000],
        [ 7.9102, 15.6328, 26.6250,  ..., 11.4531,  5.3945, 18.6562],
        [ 2.7266,  0.0000,  0.0000,  ...,  0.0000, 26.1250,  0.0000],
        ...,
        [ 0.4316, 75.2500,  0.0000,  ..., 26.2812,  0.0000,  0.0000],
        [ 6.9570,  1.1260,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [27.6406, 26.9531, 22.9375,  ..., 13.5625,  6.0391, 21.6406]],
       device='cuda:0', dt

In [76]:
# -----------------------------------------------------------------------------
# Performance Benchmark
# IMPORTANT: DO NOT CHANGE THIS CODE.
# THIS IS THE EXACT CODE THAT WILL BE USED TO GRADE YOUR IMPLEMENTATION.
# ANY CHANGES TO THIS CODE (INCLUDING DIMENSIONS, REPEATS, etc.)
# WILL CAUSE YOU TO HAVE DIFFERENT SPEEDUP RESULTS.
# -----------------------------------------------------------------------------
M = 2048
K = 2048
N = 2048

# KEEP THESE MATRICES IN FP16. FP32 WILL NOT PROVIDE ACCURATE RESULTS
A = torch.randn((M, K), device="cuda", dtype=torch.float16)
B = torch.randn((K, N), device="cuda", dtype=torch.float16)
C = torch.randn((M, N), device="cuda", dtype=torch.float16)

# warmup
_ = matmul_add_relu_fp16(A, B, C)
_ = reference_matmul_add_relu(A, B, C)

REPEATS = 5000

# time your implementation
print("Triton implementation")
torch.cuda.synchronize()
start = time.perf_counter()
for _ in range(REPEATS):
    _ = matmul_add_relu_fp16(A, B, C)
torch.cuda.synchronize()
triton_time = (time.perf_counter() - start) / REPEATS

# time pytorch
print("PyTorch implementation")
torch.cuda.synchronize()
start = time.perf_counter()
for _ in range(REPEATS):
    _ = reference_matmul_add_relu(A, B, C)
torch.cuda.synchronize()
torch_time = (time.perf_counter() - start) / REPEATS

print(f"Performance comparison for matrix multiplication ({M}x{K} @ {K}x{N}):")
print(f"Triton implementation: {triton_time*1000:.2f} ms")
print(f"PyTorch implementation: {torch_time*1000:.2f} ms")

print(f"\nSpeedup of Triton vs PyTorch: {torch_time/triton_time:.2f}x")

Triton implementation
PyTorch implementation
Performance comparison for matrix multiplication (2048x2048 @ 2048x2048):
Triton implementation: 0.92 ms
PyTorch implementation: 0.98 ms

Speedup of Triton vs PyTorch: 1.06x


In [51]:
def benchmark_kernel(BLOCK_M, BLOCK_N, BLOCK_K, iters=5000):
    #
    M, N, K = 2048

    # Create input matrices on GPU in fp16
    A = torch.randn((M, K), device='cuda', dtype=torch.float16)
    B = torch.randn((K, N), device='cuda', dtype=torch.float16)
    C = torch.randn((M, N), device='cuda', dtype=torch.float16)
    D = torch.empty((M, N), device='cuda', dtype=torch.float16)

    grid = (triton.cdiv(M, BLOCK_M), triton.cdiv(N, BLOCK_N))


    for _ in range(10):
        matmul_add_relu_kernel_fp16[grid](A, B, C, D,
                                          M, N, K,
                                          A.stride(0), A.stride(1),
                                          B.stride(0), B.stride(1),
                                          C.stride(0), C.stride(1),
                                          D.stride(0), D.stride(1),
                                          BLOCK_M, BLOCK_N, BLOCK_K)
    torch.cuda.synchronize()


    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    for _ in range(iters):
        matmul_add_relu_kernel_fp16[grid](A, B, C, D,
                                          M, N, K,
                                          A.stride(0), A.stride(1),
                                          B.stride(0), B.stride(1),
                                          C.stride(0), C.stride(1),
                                          D.stride(0), D.stride(1),
                                          BLOCK_M, BLOCK_N, BLOCK_K)
    end.record()
    torch.cuda.synchronize()


    avg_time_ms = start.elapsed_time(end) / iters
    return avg_time_ms


configs = [
    (64, 64, 16), (64, 64, 32), (64, 64, 64),
    (64, 128, 16), (64, 128, 32), (64, 128, 64),
    (128, 64, 16), (128, 64, 32), (128, 64, 64),
    (128, 128, 16), (128, 128, 32), (128, 128, 64),
    (128, 256, 16), (128, 256, 32), (128, 256, 64),
    (256, 128, 16), (256, 128, 32), (256, 128, 64),
    (256, 256, 16), (256, 256, 32), (256, 256, 64),
    (64, 256, 16), (64, 256, 32), (64, 256, 64)
]
results = {}
print("Running grid search over tiling parameters:")
for config in configs:
    BLOCK_M, BLOCK_N, BLOCK_K = config
    time_ms = benchmark_kernel(BLOCK_M, BLOCK_N, BLOCK_K)
    results[config] = time_ms
    print(f"Config {config}: {time_ms:.3f} ms per iteration")

best_config = min(results, key=results.get)
print("\nBest configuration:", best_config, "with average runtime:",
      f"{results[best_config]:.3f} ms per iteration")

Running grid search over tiling parameters:
Config (64, 64, 16): 1.906 ms per iteration
Config (64, 64, 32): 1.654 ms per iteration
Config (64, 64, 64): 1.496 ms per iteration
Config (64, 128, 16): 1.171 ms per iteration
Config (64, 128, 32): 1.191 ms per iteration
Config (64, 128, 64): 1.078 ms per iteration
Config (128, 64, 16): 1.348 ms per iteration
Config (128, 64, 32): 1.269 ms per iteration
Config (128, 64, 64): 1.169 ms per iteration
Config (128, 128, 16): 0.995 ms per iteration
Config (128, 128, 32): 0.817 ms per iteration
Config (128, 128, 64): 0.877 ms per iteration
Config (128, 256, 16): 2.069 ms per iteration
Config (128, 256, 32): 2.131 ms per iteration
Config (128, 256, 64): 43.371 ms per iteration
Config (256, 128, 16): 1.867 ms per iteration
Config (256, 128, 32): 44.440 ms per iteration
Config (256, 128, 64): 50.572 ms per iteration
Config (256, 256, 16): 41.918 ms per iteration
Config (256, 256, 32): 47.524 ms per iteration
Config (256, 256, 64): 50.021 ms per iterat